# Hello, FABRIC: Create your first FABRIC slice

## Configure the Environment

Set the environment variable that will be used by this notebook. If you are using the FABRIC, JupyterHub some of the environment will be automatically configured for you.  You will only need to set your bastion username, upload your bastion private key, and set the path to where you put your bastion private key. Your bastion username and private key should already be in your possession.  If you do not have a bastion username and private key, please contact the FABRIC admins using the [FABRIC User Forum](https://learn.fabric-testbed.net/forums/) 

If you are using the FABRIC API outside of the JupyterHub you will need to configure all of the environment variables. Defaults below will be correct in many situations but you will need to confirm your configuration.  If you have questions about this configuration, please contact the FABRIC admins using the [FABRIC User Forum](https://learn.fabric-testbed.net/forums/) 

More information about accessing your experiments through the FABRIC bastion hosts can be found [here](https://learn.fabric-testbed.net/knowledge-base/logging-into-fabric-vms/).

In [ ]:
import os

# If you are using the FABRIC JupyterHub, the following three evnrionment vars
# were automatically provided when you logged in.
#os.environ['FABRIC_CREDMGR_HOST']='cm.fabric-testbed.net'
#os.environ['FABRIC_ORCHESTRATOR_HOST']='orchestrator.fabric-testbed.net'
#os.environ['FABRIC_TOKEN_LOCATION']=os.environ['HOME']+'/work/fabric_token.json'

# Bastion IPs
os.environ['FABRIC_BASTION_HOST'] = 'bastion-1.fabric-testbed.net'

# Set your Bastion username and private key
os.environ['FABRIC_BASTION_USERNAME']='pruth_0031379841'
os.environ['FABRIC_BASTION_KEY_LOCATION']=os.environ['HOME']+'/work/fabric_bastion_key'

# Set the keypair FABRIC will install in your slice. 
os.environ['FABRIC_SLICE_PRIVATE_KEY_FILE']=os.environ['HOME']+'/.ssh/id_rsa'
os.environ['FABRIC_SLICE_PUBLIC_KEY_FILE']=os.environ['HOME']+'/.ssh/id_rsa.pub'

# If your slice private key uses a passphrase, set the passphrase
#from getpass import getpass
#print('Please input private key passphrase. Press enter for no passphrase.')
#os.environ['FABRIC_SLICE_PRIVATE_KEY_PASSPHRASE']=getpass()

## Setup the Experiment

#### Import the FABRIC API

In [ ]:
import json
import traceback
from fabrictestbed_extensions.fablib.fablib import fablib

#### Create the FABRIC Proxies

The FABRIC API is used via proxy objects that manage connections to the control framework.  

#### (Optional) Query Available Resources

This optional command queries the FABRIC services to find the available resources. It may be useful for finding a site with available capacity.

In [ ]:
try:
    available_resources = fablib.get_available_resources()
    print(f"Available Resources: {available_resources}")
except Exception as e:
    print(f"Error: {e}")

## Create the Experiment Slice

#### Configure the Experiment Parameters



In [ ]:
slice_name = 'MySlice'
site = fablib.get_random_site()
print(f"Site: {site}")
node1_name = 'Node1'
node2_name = 'Node2'
image = 'default_ubuntu_20'
node1_nic_name = 'NIC1'
node2_nic_name = 'NIC2'
network_name = 'NET1'

### Create Slice

<img src="./figs/SingleNode.png" width="20%"><br>

Create a single node with basic compute capabilities. The submit function will block until the node is ready and will display a progress bar.


In [ ]:
try:
    #Create Slice
    slice = fablib.new_slice(slice_name)

    # Add node
    node1 = slice.add_node(name=node1_name, site=site, image=image)
    iface1 = node1.add_component(model='NIC_Basic', name=node1_nic_name).get_interfaces()[0]
    
    # Add node
    node2 = slice.add_node(name=node2_name, site=site, image=image)
    iface2 = node2.add_component(model='NIC_Basic', name=node2_nic_name).get_interfaces()[0]
    
    # Network
    net1 = slice.add_l3network(name=network_name, interfaces=[iface1, iface2], type='IPv4')
    
    #Submit Slice Request
    slice_id = slice.submit()
    
except Exception as e:
    print(f"{e}")

### Print the Node's Attributes

Each node in the slice has a set of get functions that return the node's attributes.

In [ ]:
try:
    slice = fablib.get_slice(slice_id=slice_id)
    
    for node in slice.get_nodes():
        print(f"{node}")
    
except Exception as e:
    print(f"Fail: {e}")
    traceback.print_exc()

In [ ]:
try:
    network = slice.get_network(name=network_name)
    network_available_ips = network.get_available_ips()
    print(f"{network}")
    
except Exception as e:
    print(f"Exception: {e}")

In [ ]:
try:
    node1 = slice.get_node(node1_name)
    node1_iface = node1.get_interface(network_name=network_name)
    
    node1_address = network_available_ips.pop(0)
    node1_iface.ip_addr_add(addr=node1_address, subnet=network.get_subnet())
    #node1.ip_route_add(subnet=network.get_subnet(), gateway=network.get_gateway())
    
    stdout, stderr = node1.execute(f'ip addr show {node1_iface.get_os_interface()}')
    print (stdout)
    
    stdout, stderr = node1.execute(f'ip route list')
    print (stdout)
    
except Exception as e:
    print(f"Fail: {e}")
    traceback.print_exc()

In [ ]:
try:
    node2 = slice.get_node(node2_name)
    node2_iface = node2.get_interface(network_name=network_name)
    
    node2_address = network_available_ips.pop(0)
    node2_iface.ip_addr_add(addr=node2_address, subnet=network.get_subnet())
    #node2.ip_route_add(subnet=network.get_subnet(), gateway=network.get_gateway())
    
    stdout, stderr = node2.execute(f'ip addr show {node1_iface.get_os_interface()}')
    print (stdout)
    
    stdout, stderr = node2.execute(f'ip route list')
    print (stdout)
    
except Exception as e:
    print(f"Fail: {e}")
    traceback.print_exc()


## Now let's start a kubernetes cluster on the two nodes. First node is the "control plane". Second node is the "worker".

We follow the instructions that we have here: https://github.com/apache/openwhisk-deploy-kube/blob/master/docs/k8s-diy-ubuntu.md

In [ ]:
try:
    file_attributes = node1.upload_file(local_file_path="config_control_plane.sh", remote_file_path="config_control_plane.sh")
    
    stdout, stderr = node1.execute(f"chmod +x config_control_plane.sh && ./config_control_plane.sh")

except Exception as e:
    print(f"Exception: {e}")
 

## The setup part is complete. Now we start running the kubernetes cluster.

In [ ]:
try:
    file_attributes = node1.upload_file(local_file_path="start_control_plane.sh", remote_file_path="start_control_plane.sh")
    
    stdout, stderr = node1.execute(f"chmod +x start_control_plane.sh && ./start_control_plane.sh {network.get_subnet()} {node1_address}")
except Exception as e:
    print(f"Exception: {e}")
 

## Node status is "Ready".

# Client side setup.

In [ ]:
try:
    file_attributes = node2.upload_file(local_file_path="config_worker_node.sh", remote_file_path="config_worker_node.sh")
    
    stdout, stderr = node2.execute(f"chmod +x config_worker_node.sh && ./config_worker_node.sh")
except Exception as e:
    print(f"Exception: {e}")
 

## The setup part is complete. Now we need to do the join command. We do it like this:

As a note. When we do "kubeadm init" in the server, it prints the join command for us. What we do is that we copy it and paste it in the client. And don't forget to add sudo.

In [ ]:
try:
    file_attributes = node2.upload_file(local_file_path="start_worker_node.sh", remote_file_path="start_worker_node.sh")
    
    stdout, stderr = node2.execute(f"chmod +x start_worker_node.sh && ./start_worker_node.sh {node1_address}")
except Exception as e:
    print(f"Exception: {e}")
 

## Let's check back at the server.

In [ ]:
try:
    stdout, stderr = node1.execute(" kubectl get nodes")
    print(f"stdout: {stdout}")
    print(f"stderr: {stderr}")
except Exception as e:
    print(f"Exception: {e}")
 

## Node has successfully joined the cluster.

# Deploying a hello world application.

## First, we pull a hello world image and create a "deployment".

In [ ]:
try:
    stdout, stderr = node1.execute("kubectl create deployment kubernetes-bootcamp --image=gcr.io/google-samples/kubernetes-bootcamp:v1")
    print(f"stdout: {stdout}")
    print(f"stderr: {stderr}")
except Exception as e:
    print(f"Exception: {e}")
 

## Now let's do some status commands.

In [ ]:
try:
    stdout, stderr = node1.execute("kubectl get pods --all-namespaces")
    print(f"stdout: {stdout}")
    print(f"stderr: {stderr}")
except Exception as e:
    print(f"Exception: {e}")

## Pick the correct pod name and run the command below.

In [ ]:

try:
    stdout, stderr = node1.execute("kubectl describe pod kubernetes-bootcamp-65d5b99f84-brrvp")
    print(f"stdout: {stdout}")
    print(f"stderr: {stderr}")
except Exception as e:
    print(f"Exception: {e}")

## The next thing we need to do is to create what is called a "service".

We are going to use it to expose the deployment to the outside, through a port, which is 8080. Like this:

Note that the service itself will still need to be exposed. There's another "expose" step that we need to make.

In [ ]:
try:
    stdout, stderr = node1.execute('kubectl expose deployment/kubernetes-bootcamp --type="ClusterIP" --port 8080')
    print(f"stdout: {stdout}")
    print(f"stderr: {stderr}")
except Exception as e:
    print(f"Exception: {e}")

## Let's check if the service was created.

In [ ]:
try:
    stdout, stderr = node1.execute("kubectl get service kubernetes-bootcamp")
    print(f"stdout: {stdout}")
    print(f"stderr: {stderr}")
except Exception as e:
    print(f"Exception: {e}")

## Finally, we need to run a port forwarding command in order to expose the service to the outside.

Modify the --address flag. Use the "CLUSTER-IP" that is output form the command above.

In [ ]:
try:
    stdout, stderr = node1.execute("kubectl port-forward --address 10.101.194.170 service/kubernetes-bootcamp 8080:8080 > /dev/null 2>&1 &")
    print(f"stdout: {stdout}")
    print(f"stderr: {stderr}")
except Exception as e:
    print(f"Exception: {e}")

## Now our application should finally be visible. Let's test the deployment on the control plane machine itself.

In [ ]:
try:
    stdout, stderr = node1.execute("curl 10.101.194.170:8080")
    print(f"stdout: {stdout}")
    print(f"stderr: {stderr}")
except Exception as e:
    print(f"Exception: {e}")

## Delete Slice

Please delete your slicd when you are done with your experiment.

In [ ]:
try:
    slice = fablib.get_slice(slice_name)
    slice.delete()
except Exception as e:
    print(f"Fail: {e}")